# Demos for Talk Working with Large Models in ONNX IR

In [1]:
# Prepare environment

# %pip install --upgrade onnxscript onnx-ir onnx-safetensors model-explorer-onnx onnxruntime

## Demo 1: Safetensors in ONNX

**Q1: Is there a way to use the safetensors format as an external data format for ONNX?**

**A1:** Yes. The data is contiguous, row-major, and little-endian (same as ONNX). Data offset can be found by parsing the json header.

<img src="resources/safetensors-format.svg" width="500"/>

Image source: https://huggingface.co/docs/safetensors/en/index

**Q2: How do we do it efficiently?**

**A2:** Use onnx_ir to replace the tensors.

In [2]:
import onnx_ir as ir

model = ir.load("resources/model.textproto")
print(model)

<
    ir_version=10,
    opset_imports={'': 21},
    producer_name='onnx-safetensors-example',
    producer_version=None,
    domain=None,
    model_version=None,
>
graph(
    name=SimpleGraph,
    inputs=(
        %"input"<FLOAT,[1,3]>
    ),
    outputs=(
        %"output"<FLOAT,[1,3]>
    ),
    initializers=(
        %"weights"<FLOAT,[3]>{TensorProtoTensor<FLOAT,[3]>(array([1., 2., 3.], dtype=float32), name='weights')}
    ),
) {
    0 |  # :anonymous_node:128221484870704
         %"output"<FLOAT,[1,3]> ⬅️ ::Add(%"input", %"weights"{[1.0, 2.0, 3.0]})
    return %"output"<FLOAT,[1,3]>
}




### Loading tensors from a safetensors file into an ONNX model

Use `load_file_as_external_data` to load safetensors as external data and replace weights in the model

In [5]:
import onnx_safetensors
import onnx

model_with_external_data = onnx_safetensors.load_file_as_external_data(
    model,
    "resources/weights.safetensors",  # weights containing [4, 5, 6]
)

print(onnx.printer.to_text(ir.to_proto(model_with_external_data)))

<
   ir_version: 10,
   opset_import: ["" : 21],
   producer_name: "onnx-safetensors-example"
>
SimpleGraph (float[1,3] input) => (float[1,3] output) 
   <float[3] weights = ["location": "resources/weights.safetensors", "offset": "72", "length": "12"], float[3] weights>
{
   output = Add (input, weights)
}


### Using safetensors as external data for ONNX

We can similarly save external data file from an ONNX model to safetensors. By storing the tensor dtype in ONNX file, we can even use types safetensors doesn't yet support, like INT4.

You can read more at https://github.com/justinchuby/onnx-safetensors/blob/main/examples/tutorial.ipynb

### Inference with ONNX Runtime

In [7]:
import numpy as np
import onnxruntime as ort

ir.save(model_with_external_data, "model_with_external_data.onnx")
session = ort.InferenceSession("model_with_external_data.onnx")
output = session.run(None, {"input": np.array([[1.0, 2.0, 3.0]], dtype=np.float32)})
print("[[1.0, 2.0, 3.0]] + [4, 5, 6] =", output)

[[1.0, 2.0, 3.0]] + [4, 5, 6] = [array([[5., 7., 9.]], dtype=float32)]


## Demo 2: Fusion in onnxscript for ONNX Runtime

In [8]:
import onnx_ir as ir
import onnxscript.rewriter.ort_fusions

# Load the SmolLM model [show model]
smollm_model = ir.load("resources/model_SmolLM17b_2Layer-onnx_dynamo-ir-d1rt1.onnx")

# Run optimizations for ONNX Runtime
onnxscript.rewriter.ort_fusions.optimize_for_ort(smollm_model, debug=True)
ir.save(
    smollm_model,
    "model_SmolLM17b_optimized.onnx",
    external_data="model_SmolLM17b_optimized.onnx.data",
)

Applied 1 of general pattern rewrite rules.
Rule: Anonymous Rule
--------------------------------------------------------------------------------
Status: NO_MATCH
Graph matching failed: Value embedding is not a constant, expecting 0.5.
Failure at or around nodes/values:
Node: 'node_Mul_337'


%"mul_169"<FLOAT,> ⬅️ ::Mul(%"embedding", %"rsqrt")

Matched nodes:
Node: 'node_Mul_337'


%"mul_169"<FLOAT,> ⬅️ ::Mul(%"embedding", %"rsqrt")

Node: 'node_Mul_338'


...

Node: 'node_Cast_57'


%"convert_element_type_default"<FLOAT,> ⬅️ ::Cast(%"gt") {to=1}

Matched nodes:
Node: 'node_Mul_58'


%"mul_16"<FLOAT,> ⬅️ ::Mul(%"triu", %"convert_element_type_default")

--------------------------------------------------------------------------------
No matches found.


In [9]:
# !onnxvis resources/model_SmolLM17b_2Layer-onnx_dynamo-ir-d1rt1.onnx,model_SmolLM17b_optimized.onnx

# import model_explorer

# model_explorer.visualize(
#     [
#         "resources/model_SmolLM17b_2Layer-onnx_dynamo-ir-d1rt1.onnx",
#         "model_SmolLM17b_optimized.onnx",
#     ],
#     extensions=["model_explorer_onnx"],
# )


## Demo 3: Putting it together

1. Build a model with the tape module.
2. Replace some initializers
3. Build a pass to modify the model (merge QKV weights)
4. Use rewriter to do the same thing
5. Show it on model explorer


### 1. Build a model with the tape module.

In [10]:
import numpy as np


def build_model() -> ir.Model:
    R"""
      Input Embeddings  [batch_size, seq_len, hidden_size]
          |
          V
      [Linear Layer: W_QKV]  <-- Single merged weight matrix  [hidden_size, 3 * hidden_size]
          |
          V
      [QKV Tensor]  [batch_size, seq_len, 3 * hidden_size]
       /    |    \
      /     |     \
     V      V      V
    Query  Key   Value    <-- Q, K, V are now derived by splitting the QKV Tensor
    # Each of these has shape [batch_size, seq_len, hidden_size]
        \   |   /
         \  |  /
     [Scaled Dot-Product Attention]
                |
                V
        [Attention Output]  [batch_size, seq_len, hidden_size]
    """
    batch_size = 2
    seq_len = 3
    hidden_size = 16

    # Initializer the Tape. It is simply a recorder of operations and initializers.
    tape = ir.tape.Tape()

    # Create initializers
    q_weight = tape.initializer(
        ir.tensor(np.random.rand(hidden_size, hidden_size).astype(np.float32)),
        name="q_weight",
    )
    k_weight = tape.initializer(
        ir.tensor(np.random.rand(hidden_size, hidden_size).astype(np.float32)),
        name="k_weight",
    )
    v_weight = tape.initializer(
        ir.tensor(np.random.rand(hidden_size, hidden_size).astype(np.float32)),
        name="v_weight",
    )

    # Create graph inputs
    input = ir.Value(
        name="input",
        type=ir.TensorType(ir.DataType.FLOAT),
        shape=ir.Shape([batch_size, seq_len, hidden_size]),
    )

    query = tape.op("MatMul", inputs=[input, q_weight])
    key = tape.op("MatMul", inputs=[input, k_weight])
    value = tape.op("MatMul", inputs=[input, v_weight])
    attention_output = tape.op(
        "Attention",
        inputs=[query, key, value],
        attributes={"q_num_heads": 1, "kv_num_heads": 1},
    )
    attention_output.shape = ir.Shape([batch_size, seq_len, hidden_size])
    attention_output.type = ir.TensorType(ir.DataType.FLOAT)

    model = ir.Model(
        graph=ir.Graph(
            inputs=[input],
            outputs=[attention_output],
            nodes=tape.nodes,
            initializers=tape.initializers,
            opset_imports={"": 23},
            name="main_graph",
        ),
        ir_version=10,
    )

    return model

### 2. Replace some initializers

In [23]:
model = build_model()
model.graph.initializers["q_weight"].const_value.display()

Tensor<FLOAT,[16,16]>(array([[0.48794413, 0.37564784, 0.646931  , 0.1451041 , 0.00512798, 0.33926377, 0.60621136, 
0.45370337, 0.33162656, 0.190542  , 0.21808352, 0.19473173, 0.20885149, 0.6684639 , 0.52152795, 0.86171544], 
[0.49663422, 0.6925561 , 0.11541724, 0.20630342, 0.98236555, 0.81081724, 0.8174751 , 0.43920037, 0.3933322 , 
0.09877441, 0.89563113, 0.44094598, 0.14066125, 0.11981746, 0.59848124, 0.25461426], [0.39299694, 0.03681421, 
0.5066049 , 0.2512647 , 0.20144394, 0.16203453, 0.7727595 , 0.8401172 , 0.05992422, 0.06714439, 0.52033174, 
0.00171337, 0.25144818, 0.6825386 , 0.08284873, 0.15570474], [0.21498488, 0.14031397, 0.7428406 , 0.43695948, 
0.16902204, 0.22256295, 0.6634157 , 0.58190733, 0.6638061 , 0.16951245, 0.710005  , 0.22019322, 0.59997475, 
0.2529446 , 0.3284442 , 0.5703782 ], [0.5006518 , 0.87583464, 0.49306166, 0.15325034, 0.12398469, 0.6065498 , 
0.12014221, 0.45753038, 0.85208696, 0.79901165, 0.18765211, 0.85117286, 0.958803  , 0.74697036, 0.09380716, 
0.92195207], [0.42951605, 0.42845255, 0.98265636, 0.5173211 , 0.81635123, 0.3218149 , 0.7491496 , 0.69889665, 
0.06406955, 0.38574502, 0.02530624, 0.11623917, 0.5011012 , 0.3441589 , 0.2059734 , 0.31884694], [0.577229  , 
0.57820284, 0.9819241 , 0.10528568, 0.05719717, 0.81067747, 0.22649768, 0.22390506, 0.40414968, 0.38568512, 
0.02173878, 0.37844756, 0.86168504, 0.36159396, 0.4253754 , 0.50405705], [0.99689776, 0.69047946, 0.186792  , 
0.5388528 , 0.9996349 , 0.6803643 , 0.9179063 , 0.6334719 , 0.5604079 , 0.7733181 , 0.616506  , 0.88604367, 
0.44138384, 0.21676436, 0.7513765 , 0.00272758], [0.84101576, 0.9317419 , 0.2155534 , 0.1364216 , 0.80795544, 
0.3515474 , 0.7475373 , 0.42448542, 0.06523927, 0.3390376 , 0.38688397, 0.95620245, 0.03685849, 0.06425347, 
0.9992754 , 0.5059519 ], [0.04931008, 0.43877056, 0.60118806, 0.650699  , 0.619442  , 0.74798703, 0.090756  , 
0.29690343, 0.83723223, 0.5402168 , 0.8455248 , 0.74905574, 0.5346829 , 0.2916724 , 0.46183303, 0.42637402], 
[0.93773186, 0.02240606, 0.15469854, 0.5504482 , 0.70858437, 0.55014485, 0.6785477 , 0.18265791, 0.22148678, 
0.48467344, 0.17710513, 0.0180437 , 0.4140666 , 0.30774   , 0.9378072 , 0.2552333 ], [0.8354899 , 0.10369226, 
0.8844558 , 0.21960302, 0.1703607 , 0.27043805, 0.8281073 , 0.02655184, 0.23483448, 0.8989429 , 0.607386  , 
0.08340706, 0.9285259 , 0.09208788, 0.73677915, 0.7331335 ], [0.08062557, 0.18003123, 0.9209848 , 0.17361055, 
0.22648598, 0.6863814 , 0.40177515, 0.45734933, 0.23541933, 0.7086931 , 0.37085983, 0.9552729 , 0.64784926, 
0.80490404, 0.88512075, 0.29762906], [0.02504105, 0.9105067 , 0.49034315, 0.7740004 , 0.9097915 , 0.6810732 , 
0.4313761 , 0.91411716, 0.5457693 , 0.771856  , 0.5341072 , 0.13470322, 0.02397172, 0.78177917, 0.62038386, 
0.44721615], [0.5375618 , 0.5746843 , 0.10875943, 0.4944015 , 0.66971284, 0.62009263, 0.6972071 , 0.96035165, 
0.11824181, 0.5843182 , 0.09219486, 0.9571733 , 0.40124694, 0.35149458, 0.40311018, 0.6929439 ], [0.89384824, 
0.10437008, 0.84827054, 0.5845285 , 0.6556607 , 0.00616558, 0.89488786, 0.5080533 , 0.33442673, 0.8220109 , 
0.12484946, 0.9344894 , 0.5621439 , 0.6137998 , 0.46775892, 0.19139378]], dtype=float32), name=None)

Min: 0.0017133677611127496, Max: 0.9996349215507507, NaN count: 0, Inf count: 0
Sparsity (abs<1e-06): 0.00
Histogram:
       9 ┼                ╭╮
       8 ┤                ││
       7 ┤╭╮      ╭╮      ││
       6 ┤││      ││      ││                                        ╭╮
       4 ┼╯│    ╭─╯│ ╭──╮╭╯╰╮╭╮        ╭╮╭╮╭──╮ ╭─╮ ╭╮ ╭╮╭─╮  ╭──╮  ││╭╮ ╭╮ ╭─╮╭─╮╭╮ ╭╮
       3 ┤ │ ╭──╯  │╭╯  ╰╯  │││ ╭╮ ╭──╮│││╰╯  │ │ ╰─╯│ │││ │  │  ╰╮ ││││ │╰─╯ ││ ││╰╮││╭─
       2 ┤ ╰╮│     ╰╯       │││ ││╭╯  ╰╯╰╯    │╭╯    ╰─╯╰╯ │╭─╯   │╭╯│││ │    ││ ╰╯ ╰╯││
       1 ┤  ╰╯              ││╰╮│╰╯           ╰╯           ╰╯     ││ ╰╯╰─╯    ││      ││
       0 ┤                  ╰╯ ╰╯                                 ╰╯          ╰╯      ╰╯
     0.0017  0.0890  0.1888  0.2761  0.3759  0.4633  0.5630  0.6504  0.7502  0.8375  0.93

In [24]:
hidden_size = 16

model.graph.initializers["q_weight"].const_value = ir.tensor(
    np.zeros((hidden_size, hidden_size)).astype(np.float32), name="q_weight"
)
model.graph.initializers["q_weight"].const_value.display()

Tensor<FLOAT,[16,16]>(array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], [0., 0., 0., 0., 0.,
0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], [0.,
0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 
0., 0., 0.], [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], [0., 0., 0., 0., 0., 0., 0., 0., 0.,
0., 0., 0., 0., 0., 0., 0.], [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], [0., 0., 0., 0., 0.,
0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], [0.,
0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 
0., 0., 0.], [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], [0., 0., 0., 0., 0., 0., 0., 0., 0.,
0., 0., 0., 0., 0., 0., 0.], [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], [0., 0., 0., 0., 0.,
0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]], dtype=float32), name='q_weight')

Min: 0.0, Max: 0.0, NaN count: 0, Inf count: 0
Sparsity (abs<1e-06): 1.00
Histogram:
     256 ┼                                       ╭╮
     224 ┤                                       ││
     192 ┤                                       ││
     160 ┤                                       ││
     128 ┤                                       ││
      96 ┤                                       ││
      64 ┤                                       ││
      32 ┤                                       ││
       0 ┼───────────────────────────────────────╯╰──────────────────────────────────────
    -0.5000  -0.4000  -0.3000  -0.1875  -0.0875  0.0250  0.1125  0.2125  0.3000  0.4000  0.5000

### 3. Build a pass to modify the model (merge QKV weights)

In [25]:
# Now we want to combine the weights into a single QKV weight matrix

for node in model.graph:
    if node.op_type != "Attention":
        continue

    # Find the weights for Q, K, V
    input_val = node.inputs[0].producer().inputs[0]
    q_weight_val = node.inputs[0].producer().inputs[1]
    k_weight_val = node.inputs[1].producer().inputs[1]
    v_weight_val = node.inputs[2].producer().inputs[1]
    assert q_weight_val.const_value is not None and q_weight_val.is_initializer()
    assert k_weight_val.const_value is not None and q_weight_val.is_initializer()
    assert v_weight_val.const_value is not None and q_weight_val.is_initializer()

    qkv_weight = ir.Value(
        name="qkv_weight",
        type=ir.TensorType(ir.DataType.FLOAT),
        shape=ir.Shape([hidden_size, 3 * hidden_size]),
        const_value=ir.tensor(
            np.concatenate(
                [
                    q_weight_val.const_value,
                    k_weight_val.const_value,
                    v_weight_val.const_value,
                ],
                axis=1,
            )
        ),
    )
    # Create a new MatMul node uses the combined Q, K, V weights
    combined_matmul_node = ir.node("MatMul", inputs=[input_val, qkv_weight])
    new_qkv = ir.node(
        "Split",
        inputs=combined_matmul_node.outputs,
        attributes={
            "axis": 2,
            "num_outputs": 3,
        },
        num_outputs=3,
    )
    # Add the new node to the graph and register the new initializer
    node.prepend((combined_matmul_node, new_qkv))
    # Reconnect the Attention node to use the new QKV outputs
    ir.convenience.replace_all_uses_with(
        node.inputs,
        new_qkv.outputs,
    )
    # Add the new initializer to the graph
    model.graph.register_initializer(qkv_weight)


In [30]:
print("Combined QKV model:")
print(model)
print("⚠️ There are still unused nodes and initializers.")

Combined QKV model:
<
    ir_version=10,
    opset_imports={'': 23},
    producer_name=None,
    producer_version=None,
    domain=None,
    model_version=None,
>
graph(
    name=main_graph,
    inputs=(
        %"input"<FLOAT,[2,3,16]>
    ),
    outputs=(
        %"val_3"<FLOAT,[2,3,16]>
    ),
    initializers=(
        %"q_weight"<FLOAT,[16,16]>{Tensor(...)},
        %"k_weight"<FLOAT,[16,16]>{Tensor(...)},
        %"v_weight"<FLOAT,[16,16]>{Tensor(...)},
        %"qkv_weight"<FLOAT,[16,48]>{Tensor(...)}
    ),
) {
    0 |  # node_MatMul_0
         %"val_0"<?,?> ⬅️ ::MatMul(%"input", %"q_weight"{...})
    1 |  # node_MatMul_1
         %"val_1"<?,?> ⬅️ ::MatMul(%"input", %"k_weight"{...})
    2 |  # node_MatMul_2
         %"val_2"<?,?> ⬅️ ::MatMul(%"input", %"v_weight"{...})
    3 |  # node_MatMul_4
         %"val_4"<?,?> ⬅️ ::MatMul(%"input", %"qkv_weight"{...})
    4 |  # node_Split_5
         %"val_5"<?,?>, %"val_6"<?,?>, %"val_7"<?,?> ⬅️ ::Split(%"val_4") {axis=2, num_outputs=3}

### 3.5. Run passes

In [31]:
import onnx_ir.passes.common as common_passes

common_passes.RemoveUnusedNodesPass()(model)

print("Model after removing unused nodes:")
print(model)

Model after removing unused nodes:
<
    ir_version=10,
    opset_imports={'': 23},
    producer_name=None,
    producer_version=None,
    domain=None,
    model_version=None,
>
graph(
    name=main_graph,
    inputs=(
        %"input"<FLOAT,[2,3,16]>
    ),
    outputs=(
        %"val_3"<FLOAT,[2,3,16]>
    ),
    initializers=(
        %"qkv_weight"<FLOAT,[16,48]>{Tensor(...)}
    ),
) {
    0 |  # node_MatMul_4
         %"val_4"<?,?> ⬅️ ::MatMul(%"input", %"qkv_weight"{...})
    1 |  # node_Split_5
         %"val_5"<?,?>, %"val_6"<?,?>, %"val_7"<?,?> ⬅️ ::Split(%"val_4") {axis=2, num_outputs=3}
    2 |  # node_Attention_3
         %"val_3"<FLOAT,[2,3,16]> ⬅️ ::Attention(%"val_5", %"val_6", %"val_7") {q_num_heads=1, kv_num_heads=1}
    return %"val_3"<FLOAT,[2,3,16]>
}




In [21]:
common_passes.ShapeInferencePass()(model)

print("Model after shape inference:")
print(model)

Model after shape inference:
<
    ir_version=10,
    opset_imports={'': 23},
    producer_name=None,
    producer_version=None,
    domain=None,
    model_version=None,
>
graph(
    name=main_graph,
    inputs=(
        %"input"<FLOAT,[2,3,16]>
    ),
    outputs=(
        %"val_3"<FLOAT,[2,3,16]>
    ),
    initializers=(
        %"qkv_weight"<FLOAT,[16,48]>{Tensor(...)}
    ),
) {
    0 |  # node_MatMul_4
         %"val_4"<FLOAT,[2,3,48]> ⬅️ ::MatMul(%"input", %"qkv_weight"{...})
    1 |  # node_Split_5
         %"val_5"<FLOAT,[2,3,16]>, %"val_6"<FLOAT,[2,3,16]>, %"val_7"<FLOAT,[2,3,16]> ⬅️ ::Split(%"val_4") {axis=2, num_outputs=3}
    2 |  # node_Attention_3
         %"val_3"<FLOAT,[2,3,16]> ⬅️ ::Attention(%"val_5", %"val_6", %"val_7") {q_num_heads=1, kv_num_heads=1}
    return %"val_3"<FLOAT,[2,3,16]>
}




### 4. Use rewriter to do the same thing

In [ ]:
# Now use the rewriter

import onnxscript.rewriter as rewriter


class CombineQKVWeights(rewriter.pattern.RewriteRuleClassBase):
    def pattern(cls, op, input, q_weight, k_weight, v_weight):
        # Define the pattern to match here
        q = op.MatMul(input, q_weight)
        k = op.MatMul(input, k_weight)
        v = op.MatMul(input, v_weight)
        return op.Attention(q, k, v, q_num_heads=1, kv_num_heads=1)

    def rewrite(cls, op, input: ir.Value, q_weight: ir.Value, k_weight: ir.Value, v_weight: ir.Value):
        # Define the rewrite logic here
        qkv_weight = op.initializer(
            ir.tensor(
                np.concatenate(
                    [
                        q_weight.const_value.numpy(),
                        k_weight.const_value.numpy(),
                        v_weight.const_value.numpy(),
                    ],
                    axis=1,
                )
            ),
            name="qkv_weight",
        )
        combined_matmul = op.MatMul(input, qkv_weight)
        new_q, new_k, new_v = op.Split(
            combined_matmul, axis=2, num_outputs=3, _outputs=3
        )
        return op.Attention(new_q, new_k, new_v, q_num_heads=1, kv_num_heads=1)


model = build_model()
# Create the rewrite rule
rule = CombineQKVWeights.rule()
# Apply the rewrite rule to the model
rule.apply_to_model(model)
# Clean up and run shape inference. Note that you can use the Sequential pass to chain multiple passes together.
ir.passes.Sequential(
    # onnxscript.rewriter.RewritePass([rule]),  # You can use it as a pass as well
    common_passes.RemoveUnusedNodesPass(),
    common_passes.ShapeInferencePass(),
)(model)

print(model)

<
    ir_version=10,
    opset_imports={'': 23},
    producer_name=None,
    producer_version=None,
    domain=None,
    model_version=None,
>
graph(
    name=main_graph,
    inputs=(
        %"input"<FLOAT,[2,3,16]>
    ),
    outputs=(
        %"val_3"<FLOAT,[2,3,16]>
    ),
    initializers=(
        %"qkv_weight"<FLOAT,[16,48]>{Tensor(...)}
    ),
) {
    0 |  # node_MatMul_4
         %"val_4"<FLOAT,[2,3,48]> ⬅️ ::MatMul(%"input", %"qkv_weight"{...})
    1 |  # node_Split_5
         %"val_5"<FLOAT,[2,3,16]>, %"val_6"<FLOAT,[2,3,16]>, %"val_7"<FLOAT,[2,3,16]> ⬅️ ::Split(%"val_4") {axis=2, num_outputs=3}
    2 |  # node_Attention_6
         %"val_3"<FLOAT,[2,3,16]> ⬅️ ::Attention(%"val_5", %"val_6", %"val_7") {q_num_heads=1, kv_num_heads=1}
    return %"val_3"<FLOAT,[2,3,16]>
}




### 5. Show it on model explorer

In [35]:
ir.save(model, "merged_qkv.onnx")

In [ ]:
# !onnxvis merged_qkv.onnx

# import model_explorer

# model_explorer.visualize("merged_qkv.onnx", extensions=["model_explorer_onnx"])